In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import LeaveOneGroupOut, KFold
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define folders containing the CSV files
folder = '/content/drive/MyDrive/train_data_windows.npz'

In [5]:
# Extract test windows and labels
loaded_data = np.load(folder)
loaded_train_windows_data = loaded_data['train_windows']
loaded_train_labels_data = loaded_data['train_labels']

In [6]:
loaded_train_labels_data

array([2, 2, 2, ..., 2, 2, 2])

In [7]:
transformed_labels = np.where(loaded_train_labels_data == 1, 0, 1)

In [8]:
transformed_labels

array([1, 1, 1, ..., 1, 1, 1])

In [9]:
X_tensor = torch.tensor(loaded_train_windows_data, dtype=torch.float32)
y_tensor = torch.tensor(transformed_labels, dtype=torch.int32)

In [10]:
# Define your dataset
class HandWashDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [11]:
# Define the Transformer-based model
class InertialTransformer(nn.Module):
    def __init__(self, num_features, seq_len, num_classes=1):
        super(InertialTransformer, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=5)
        self.transformer_encoder = nn.TransformerEncoder(
        nn.TransformerEncoderLayer(d_model=64, nhead=8), num_layers=6)
        self.classifier = nn.Sequential(
            nn.Linear(9344, 64),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        #print("start")
        #print(x.shape)
        x = x.permute(0, 2, 1)
        x = self.conv1d(x)
        #print("conv")
        #print(x.shape)
        x = x.permute(2, 1, 0)  # Reshape for transformer
        x = self.transformer_encoder(x)
        #print("transformer")
        #print(x.shape)
        x = x.permute(1, 0, 2)
        #print(x.shape)
        x = x.flatten(1)  # Flatten for classifier
        #print("flatten")
        #print(x.shape)
        #x = x.transpose(1,0)
        #print("transpose")
        #print(x.shape)
        x = self.classifier(x)
        #print("classifier")
        #print(x.shape)
        #print(x)
        return x.squeeze()

In [19]:

kf = KFold(n_splits=2)
#for train_index, test_index in kf.split(X_tensor):
for fold, (train_index, val_index) in enumerate(kf.split(X_tensor)):

    X_train, X_test = X_tensor[train_index], X_tensor[val_index]
    y_train, y_test = y_tensor[train_index], y_tensor[val_index]

    train_loader = DataLoader(HandWashDataset(X_train, y_train), batch_size=64, shuffle=False)
    test_loader = DataLoader(HandWashDataset(X_test, y_test), batch_size=64, shuffle=False)


    model = InertialTransformer(num_features=6, seq_len=150, num_classes=2)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4, betas=(0.9, 0.999))

    # Training
    for epoch in range(5):
        for inputs, labels in train_loader:
            tensor_size = torch.Size([64, 150, 6])
            if inputs.size() != tensor_size:
              print("reached end")
              break
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.unsqueeze(1)
            predicted_probabilities = torch.sigmoid(outputs).round()
            #print(predicted_probabilities)
            #outputs = outputs.unsqueeze(1)
            labels = labels.unsqueeze(1)
            #print(labels)
            #probabilities = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities
            #predictions = (probabilities > 0.5).int()  # Convert to 0 and 1
            #print("model outputs")
            loss = criterion(predicted_probabilities, labels.float())
            #print(loss)
            loss.backward()
            optimizer.step()

        print(f"Fold {fold + 1}, Epoch {epoch + 1}, Loss: {loss.item()}")

    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
      for data, labels in test_loader:
        tensor_size = torch.Size([64, 150, 6])
        if data.size() != tensor_size:
          print("tes end")
          break
        outputs = model(data)
        outputs = outputs.unsqueeze(1)
        predictions = torch.sigmoid(outputs).round() # Apply sigmoid and round to get binary class
        total += labels.size(0)
        correct += (predictions.squeeze(1) == labels).sum().item()

    accuracy = correct / total
    print(f"Accuracy: {accuracy * 100:.2f}%")

reached end
Fold 1, Epoch 1, Loss: 10.9375
reached end
Fold 1, Epoch 2, Loss: 9.375
reached end
Fold 1, Epoch 3, Loss: 4.6875
reached end
Fold 1, Epoch 4, Loss: 4.6875
reached end
Fold 1, Epoch 5, Loss: 4.6875
tes end
Accuracy: 93.61%
reached end
Fold 2, Epoch 1, Loss: 18.75
reached end
Fold 2, Epoch 2, Loss: 25.0
reached end
Fold 2, Epoch 3, Loss: 29.6875
reached end
Fold 2, Epoch 4, Loss: 45.3125
reached end
Fold 2, Epoch 5, Loss: 62.5
tes end
Accuracy: 14.06%
